In [1]:
import torch
from diffusers import StableDiffusionUpscalePipeline
from moviepy.editor import VideoFileClip, ImageClip, concatenate_videoclips
from PIL import Image
import os

C:\Users\Suraj\AppData\Roaming\Python\Python311\site-packages\diffusers\models\transformers\transformer_2d.py:34: FutureWarning: `Transformer2DModelOutput` is deprecated and will be removed in version 1.0.0. Importing `Transformer2DModelOutput` from `diffusers.models.transformer_2d` is deprecated and this will be removed in a future version. Please use `from diffusers.models.modeling_outputs import Transformer2DModelOutput`, instead.
  deprecate("Transformer2DModelOutput", "1.0.0", deprecation_message)


In [2]:
# Load the pre-trained pipeline with mixed precision
pipeline = StableDiffusionUpscalePipeline.from_pretrained(
    "stabilityai/stable-diffusion-x4-upscaler",
    torch_dtype=torch.float16
)

Cannot initialize model with low cpu memory usage because `accelerate` was not found in the environment. Defaulting to `low_cpu_mem_usage=False`. It is strongly recommended to install `accelerate` for faster and less memory-intense model loading. You can do so with: 
```
pip install accelerate
```
.


Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

In [3]:
# Function to upscale a single frame
def upscale_frame(frame, pipeline, prompt):
    img = Image.fromarray(frame).convert("RGB")
    low_res_img = img.resize((640, 480))  # assuming the input frame is in SD resolution
    upscaled_image = pipeline(prompt=prompt, image=low_res_img).images[0]
    return upscaled_image

In [4]:
# Function to upscale a video
def upscale_video(input_path, output_path, pipeline, prompt, batch_size=2):
    video = VideoFileClip(input_path)
    upscaled_frames = []

    # Create directory for upscaled frames
    if not os.path.exists("frames"):
        os.makedirs("frames")

    for batch_start in range(0, int(video.duration * video.fps), batch_size):
        batch_end = min(batch_start + batch_size, int(video.duration * video.fps))
        for i in range(batch_start, batch_end):
            frame = video.get_frame(i / video.fps)
            upscaled_frame = upscale_frame(frame, pipeline, prompt)
            upscaled_frames.append(upscaled_frame)
            upscaled_frame.save(f"frames/frame_{i:04d}.png")
        torch.cuda.empty_cache()  # Clear cache after each batch

    # Create video from upscaled frames
    out_video = concatenate_videoclips(
        [ImageClip(f"frames/frame_{i:04d}.png").set_duration(1 / video.fps) for i in range(len(upscaled_frames))]
    )
    out_video.write_videofile(output_path, fps=video.fps)


In [5]:
# Define the input video path, output video path, and the prompt for the pipeline
input_video_path = "480psample.mp4"
output_video_path = "output.mp4"
prompt = "A high-quality HD version of this video."


In [2]:
# Upscale the video
upscale_video(input_video_path, output_video_path, pipeline, prompt)